In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.vgg19 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf
import PIL
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
from keras import applications
import pandas as pd
from keras.models import load_model
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import *
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os




img_width, img_height = 256, 256

TEST_DIR = '/content/val/val'
TRAIN_DIR = '/content/train/train'
VAL_DIR = '/content/test'



nb_train_samples = 3138
nb_validation_samples = 772 

batch_size = 32
epochs = 50



model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer=SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)


test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)


train_generator = train_datagen.flow_from_directory(
TRAIN_DIR,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
VAL_DIR,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")



# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')




if (os.path.exists('model.tfl')):
    model_final = load_model('model.tfl')
else:
    model_final.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    callbacks = [checkpoint, early])
    model_final.save('model.tfl')

# Train the model 



testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(img_width,img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model_final.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)

    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")